In [16]:
# 导入必要的库
import sys
import os

# 添加项目根目录到Python路径
sys.path.insert(0, os.path.abspath('../../'))

# 导入数据库相关模块
from table.TJCAUSP_ORIGIN import TJCAUSP_ORIGIN
from table.TJCAUSP import TJCAUSP
from table.TJCAUSP import TJCAUSP  # 导入新表的ORM
from sql.database import get_db_session, get_db
import pandas as pd

In [17]:
# 连接数据库并查询TJCAUSP_ORIGIN表的数据
with get_db_session() as db:
    records = db.query(TJCAUSP_ORIGIN).all()

    # 将查询结果转换为pandas DataFrame
    data = []
    for record in records:
        data.append({
            'id': record.id,
            'ORDER_NO': record.ORDER_NO,
            'ORDER_MONTH': record.ORDER_MONTH,
            'IN_MAT_MIN_WIDTH': record.IN_MAT_MIN_WIDTH,
            'IN_MAT_MAX_WIDTH': record.IN_MAT_MAX_WIDTH,
            'IN_MAT_MIN_THICK': record.IN_MAT_MIN_THICK,
            'IN_MAT_MAX_THICK': record.IN_MAT_MAX_THICK
        })

    df = pd.DataFrame(data)
    print(f"从TJCAUSP_ORIGIN表中查询到 {len(df)} 条记录")

2025-10-22 12:57:15,951 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-22 12:57:15,953 INFO sqlalchemy.engine.Engine SELECT "TJCAUSP_ORIGIN".id AS "TJCAUSP_ORIGIN_id", "TJCAUSP_ORIGIN"."ORDER_NO" AS "TJCAUSP_ORIGIN_ORDER_NO", "TJCAUSP_ORIGIN"."ORDER_MONTH" AS "TJCAUSP_ORIGIN_ORDER_MONTH", "TJCAUSP_ORIGIN"."IN_MAT_MIN_WIDTH" AS "TJCAUSP_ORIGIN_IN_MAT_MIN_WIDTH", "TJCAUSP_ORIGIN"."IN_MAT_MAX_WIDTH" AS "TJCAUSP_ORIGIN_IN_MAT_MAX_WIDTH", "TJCAUSP_ORIGIN"."IN_MAT_MIN_THICK" AS "TJCAUSP_ORIGIN_IN_MAT_MIN_THICK", "TJCAUSP_ORIGIN"."IN_MAT_MAX_THICK" AS "TJCAUSP_ORIGIN_IN_MAT_MAX_THICK" 
FROM "TJCAUSP_ORIGIN"
2025-10-22 12:57:15,954 INFO sqlalchemy.engine.Engine [cached since 1129s ago] ()
从TJCAUSP_ORIGIN表中查询到 124 条记录
2025-10-22 12:57:15,958 INFO sqlalchemy.engine.Engine ROLLBACK


In [22]:
# 取df的前10条数据，根据每条数据的最大宽度、最小宽度、最大厚度、最小厚度，正太分布生成这个区间内的宽度和厚度，然后根据这个分布生成对应的数据，每条数据随机生成8到12条，然后把新得到的df打印出来
import numpy as np

# 取df的前10条数据
df_subset = df.head(15)

# 创建一个列表来存储新生成的数据
new_data = []

for index, row in df_subset.iterrows():
    # 获取当前行的宽度和厚度范围
    min_width = row['IN_MAT_MIN_WIDTH']
    max_width = row['IN_MAT_MAX_WIDTH']
    min_thick = row['IN_MAT_MIN_THICK']
    max_thick = row['IN_MAT_MAX_THICK']
    
    # 计算均值和标准差
    width_mean = (min_width + max_width) / 2
    width_std = (max_width - min_width) / 4  # 使用范围的1/4作为标准差，以确保大部分值在范围内
    thick_mean = (min_thick + max_thick) / 2
    thick_std = (max_thick - min_thick) / 4  # 使用范围的1/4作为标准差
    
    # 随机生成8到12条数据
    num_records = np.random.randint(5, 10)
    
    for _ in range(num_records):
        # 使用正态分布生成宽度和厚度
        width = np.random.normal(width_mean, width_std)
        # 确保生成的值在原始范围内
        width = max(min_width, min(max_width, width))
        
        thickness = np.random.normal(thick_mean, thick_std)
        # 确保生成的值在原始范围内
        thickness = max(min_thick, min(max_thick, thickness))
        
        # 创建新行，保留原始字段值，仅更新宽度和厚度
        new_row = {
            'id': row['id'],
            'ORDER_NO': row['ORDER_NO'],
            'ORDER_MONTH': row['ORDER_MONTH'],
            'IN_MAT_WIDTH': width,
            'IN_MAT_THICK': thickness
        }
        new_data.append(new_row)

# 将新生成的数据转换为DataFrame
new_df = pd.DataFrame(new_data)
print(f"新生成的数据行数：{len(new_df)}")
print(new_df)



新生成的数据行数：106
     id    ORDER_NO ORDER_MONTH  IN_MAT_WIDTH  IN_MAT_THICK
0     1  G025000870      202502    885.751638      2.038891
1     1  G025000870      202502    883.453724      2.113444
2     1  G025000870      202502    887.162539      2.098953
3     1  G025000870      202502    881.116535      2.021794
4     1  G025000870      202502    889.916988      2.026222
..   ..         ...         ...           ...           ...
101  15  G025001340      202502    962.464178      1.790940
102  15  G025001340      202502    962.473862      1.806119
103  15  G025001340      202502    963.295694      1.877106
104  15  G025001340      202502    961.607573      1.825830
105  15  G025001340      202502    963.695654      1.857122

[106 rows x 5 columns]


In [24]:
# 将新生成的数据插入TJCAUSP表中
with get_db_session() as db:
    for _, row in new_df.iterrows():
        # 创建TJCAUSP对象并插入数据库
        new_record = TJCAUSP(
            ORDER_NO=row['ORDER_NO'],
            ORDER_MONTH=row['ORDER_MONTH'],
            IN_MAT_WIDTH=row['IN_MAT_WIDTH'],  # 使用生成的宽度值
            IN_MAT_THICK=row['IN_MAT_THICK']   # 使用生成的厚度值
        )
        db.add(new_record)

    # 提交事务以保存更改
    db.commit()

    print(f"成功将 {len(new_df)} 条记录插入TJCAUSP表中")

2025-10-22 13:08:37,167 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-22 13:08:37,170 INFO sqlalchemy.engine.Engine INSERT INTO "TJCAUSP" ("ORDER_NO", "ORDER_MONTH", "IN_MAT_WIDTH", "IN_MAT_THICK") VALUES (?, ?, ?, ?) RETURNING id
2025-10-22 13:08:37,171 INFO sqlalchemy.engine.Engine [cached since 675.8s ago (insertmanyvalues) 1/106 (ordered; batch not supported)] ('G025000870', '202502', 885.7516376221471, 2.0388912421585315)
2025-10-22 13:08:37,174 INFO sqlalchemy.engine.Engine INSERT INTO "TJCAUSP" ("ORDER_NO", "ORDER_MONTH", "IN_MAT_WIDTH", "IN_MAT_THICK") VALUES (?, ?, ?, ?) RETURNING id
2025-10-22 13:08:37,175 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/106 (ordered; batch not supported)] ('G025000870', '202502', 883.4537244260539, 2.113443714383332)
2025-10-22 13:08:37,175 INFO sqlalchemy.engine.Engine INSERT INTO "TJCAUSP" ("ORDER_NO", "ORDER_MONTH", "IN_MAT_WIDTH", "IN_MAT_THICK") VALUES (?, ?, ?, ?) RETURNING id
2025-10-22 13:08:37,176 INFO sqlalchemy.engine.En